<img src="./img/logos_combined.png" align="right" width="50%"></img>
<br><br><br>

# es-data-lib - Example and use cases

<p>This notebok gives a quick overview of the current state of the data access library. The purpose of the library is to reduce the barrier for third parties to create data clients on top of the data services created as part of the project.</p>
<p>The library can be downloaded from github - final location to be decided - @ <a href="https://github.com/earthserver-eu/es_data_lib">https://github.com/earthserver-eu/es_data_lib</a>
<p>The Library currently allows 4 types of data access</p>

 * Point
 * Point Timeseries
 * Area
 * Area Timeseries

In [ ]:
from es_data_lib import coverages, services
from es_data_lib.query.point import Point
from es_data_lib.query.point_timeseries import PointTimeSeries
from es_data_lib.query.area import Area
from es_data_lib.query.area_timeseries import AreaTimeseries
from es_data_lib.service import Service


import matplotlib.pyplot as plt
import numpy as np
import matplotlib



In [ ]:
service = Service("http://earthserver.pml.ac.uk/rasdaman/ows")
print service.coverages.keys()

## point example

In [ ]:
point = Point(service,55, -40,  service.coverages['OCCCI_V3_monthly_chlor_a'],date="2006-06-01T00:00:00Z")

print point.data

## point timeseries example

In [ ]:
pointTS = PointTimeSeries(service,55, -40, "2006-06-01T00:00:00Z","2006-10-01T00:00:00Z", service.coverages['OCCCI_V3_monthly_chlor_a'])
print pointTS.data

In [ ]:
# ECMWF time series test
# init ecmwf Service 
ecmwf_service = Service("http://earthserver.ecmwf.int/rasdaman/ows")

%matplotlib inline

point = PointTimeSeries(ecmwf_service,50.347472, -4.217737,  "2006-03-01T12:00:00Z", "2012-03-01T12:00:00Z", ecmwf_service.coverages['temp2m'])
plt.plot([x - 273.15 for x in point.data][0::4])
plt.show()

## area extraction example

In [ ]:
# csv output will be auto parsed into a numpy array this example then snows potting that using matplot lib
# need a clean way to remove nulls
area = Area(service, 40, 65, -30, 10, service.coverages['OCCCI_V3_monthly_chlor_a'], date="2006-09-01T00:00:00Z", output="csv")
area.data[area.data == 9.96921e+36] = None
plt.imshow(area.data, norm=matplotlib.colors.LogNorm())
plt.show()

## complex example NDWI - Water index from landsat

In [ ]:

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cStringIO

from es_data_lib.query.templates import landsat_rgb_area,ndvi,ndwi_mcfeeters
from es_data_lib.utils import create_query, web_post, web_post_file
from es_data_lib.query.query import Query
from es_data_lib.service import Service


class NDWI(Query):
    def __init__(self, service, south, north, west, east, date, coverage_id, output="png"):
        coverage = service.coverages['L8_B5_'+coverage_id]
        super(NDWI, self).__init__(service, coverage)
        self.template_params = {
            "swath_id": coverage_id,
            "south": south,
            "north": north,
            "west": west,
            "east": east,
            "date": date,
            "time_label":self.coverage_time,
            "x_label":self.x_name,
            "y_label":self.y_name,
            "output" : output
        }
        self.output = output
        self.template = ndwi_mcfeeters
        self._get_data()

    def _get_data(self):
        self.query = create_query(self)
        if self.output == "csv":
            self.data = web_post(self.wcps_url, {"query":self.query})[1:-1]
        if self.output == "netcdf":
            self.data = web_post_file(self.wcps_url, {"query":self.query})
        if self.output == "gtiff":
            self.data = web_post_file(self.wcps_url, {"query":self.query})
        if self.output == "png":
            self.data = web_post_file(self.wcps_url, {"query":self.query})


meeo_service = Service('https://eodataservice.org/rasdaman/ows')


meeo_area = NDWI(meeo_service, 4902991, 4917275, 377983, 390000, "2015-05-31T10:34:57Z" , "32631_30",output="gtiff")

with open("meeo_output.tif",'w') as output_file:
    output_file.write(meeo_area.data)


Image.open( "meeo_output.tif" ).show()